# Задание по теме 2.3.2
В этом задании вам необходимо использовать набор данных ad.csv. 
1) Датасет содержит 2 столбца: объявление (A/B) и действие (0/1).

2) Вам необходимо сформулировать гипотезу, провести ее проверку (пользуясь классическим подходом) и ответить на вопрос какое из объявлений лучше. 

Будем считать, что А - контрольная группа, В - тестовая группа, 0 - не было клика по объявлению, 1 - был клик (покупка) по объявлению.

Сформулируем гипотезы и установим уровень доверия 95%:

p(A) - коэффициент конверсии старого дизайна объявления

p(B) - коэффициент конверсии нового дизайна объявления

Н0: p(A) = p(B)

Н1: P(A) ≠ p(B)

In [55]:
import numpy as np
import pandas as pd
import scipy as sc

In [56]:
df=pd.read_csv('ad.csv')

In [57]:
df.head()

,ad_id,action
0,B,1
1,B,1
2,A,0
3,B,0
4,A,1


In [58]:
# используем функцию crosstab для создания таблицы кросс-табуляции, которая покажет:
# - сколько было кликов по объявлению в каждой группе;
# - количество просмотров объявления в каждой группе.

summary = pd.crosstab(df.ad_id,df.action, margins=True, margins_name='Total')

In [59]:
# добавляем столбец с коэффициентом конверсии (среднее значение) по каждой группе

summary['conversion_rate']=df.pivot_table(values='action', index='ad_id')
summary

action,0,1,Total,conversion_rate
ad_id,,,,
A,696,304,1000,0.304
B,628,372,1000,0.372
Total,1324,676,2000,NaN


In [60]:
# вычисляем стандартное отклонение и стандартную ошибку среднего для каждой группы

summary['std_p'] = df.pivot_table(values='action', index='ad_id', aggfunc = lambda x: np.std(x, ddof=0)).round(3)
summary['se_p'] = df.pivot_table(values='action', index='ad_id', aggfunc = lambda x: sc.stats.sem(x, ddof=0)).round(4)
summary

action,0,1,Total,conversion_rate,std_p,se_p
ad_id,,,,,,
A,696,304,1000,0.304,0.460,0.0145
B,628,372,1000,0.372,0.483,0.0153
Total,1324,676,2000,NaN,NaN,NaN


Поскольку conversion_rate(В)>conversion_rate(A), то можно сделать предварительный вывод о том, что объявление В дает большую результативность, чем объявление А.

Далее проверим, насколько разница в конверсии объявлений А и В статистически значима. Для этого используем критерий Z-статистики, при этом задав порог 5% для ошибки первого рода  (alpha=0.05).

In [61]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [62]:
# Выделяем контрольную группу А и тестовую группу В

control_res = df[df['ad_id'] == 'A']['action']
test_res = df[df['ad_id'] == 'B']['action']

In [63]:
# Считаем количество элементов в каждой группе (nobs) и количество кликов (покупок) в каждой группе (successes)

nobs = [control_res.count(), test_res.count()] 
successes = [control_res.sum(), test_res.sum()]

In [64]:
# Применяем критерий Z-статистики

z_stat, pval = proportions_ztest(successes, nobs)
print('z_stat:', z_stat.round(4))
print('p_value:', pval.round(4))

z_stat: -3.2144
p_value: 0.0013


In [65]:
alpha=0.05

In [66]:
print(pval<alpha)

True


Так как p_value < 0.05, то отклоняем нулевую гипотезу H0 и принимаем гипотезу Н1, что означает, что объявление В (новый дизайн) сработало иначе, чем объявление А (старый дизайн).

In [67]:
# Рассчитываем доверительные интервалы

(lower_con, lower_test), (upper_con, upper_test) = proportion_confint(successes, nobs=nobs, alpha=alpha)
print('Доверительный интервал для контрольной группы А:', lower_con.round(3), '-', upper_con.round(3))
print('Доверительный интервал для тестовой группы B:', lower_test.round(3), '-', upper_test.round(3))

Доверительный интервал для контрольной группы А: 0.275 - 0.333
Доверительный интервал для тестовой группы B: 0.342 - 0.402


Использование объявления В (новый дизайн) позволит поднять результативность до 40% (максимум на 10% по сравнению с объявлением А)

#### Вывод: В ходе проведенного АВ-тестирования было установлено, что использование объявления В (новый дизайн) даст большую результативность, чем использование объявления А (старый дизайн). При этом результативность повысится до 40%  (максимум на 10% по сравнению с объявлением А).